In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta
from entropy_over_time import SaE_over_time, SpE_over_time, PE_over_time, WPE_over_time, DE_over_time

In [ ]:
# Reading the synthetic rooftop PV dataset (Note that the results produced using the dummy dataset
# are likely to be different with the ones shown in the research article)

input_df = pd.read_csv("Input_data/synthetic_rooftop_data.gz", compression='gzip')
input_df.index = pd.to_datetime(input_df["t_stamp_utc"])
input_df = input_df.drop(columns=["t_stamp_utc"])
allsites = list(input_df.columns)

In [ ]:
import random

# Select 3 random rooftop PV systems
selected_site_id = dict()
num_to_select = 3
list_of_random_items = random.sample(allsites, num_to_select)

# Assign names to selected systems
first_random_system, second_random_system, third_random_system = list_of_random_items

# Create time-series for sine wave and white noise
sample_number = len(input_df.index)
amplitude = 100
mean = amplitude
std = amplitude/3
samples_in_one_day = timedelta(days=1)/input_df.index.to_series().diff().median()
input_df['WhiteNoise'] = np.random.normal(mean, std, size=sample_number)
input_df['SineWave'] = np.sin(2 * np.pi * 1 * np.arange(sample_number) / samples_in_one_day)

# Set window and series lengths
window_len = timedelta(days=60)
series_length = timedelta(days=364)
rolling_step = 1 * int(samples_in_one_day)

# Calculate entropy over time for selected systems and white noise and sine wave
WPE_dict = {system: WPE_over_time(input_df[system], series_length, window_len, step_size=rolling_step, dimension=6)
            for system in [first_random_system, second_random_system, third_random_system, 'WhiteNoise', 'SineWave']}
PE_dict = {system: PE_over_time(input_df[system], series_length, window_len, step_size=rolling_step, dimension=6)
           for system in [first_random_system, second_random_system, third_random_system, 'WhiteNoise', 'SineWave']}
DE_dict = {system: DE_over_time(input_df[system], series_length, window_len, step_size=rolling_step, dimension=5, cls=5)
           for system in [first_random_system, second_random_system, third_random_system, 'WhiteNoise', 'SineWave']}
SpE_dict = {system: SpE_over_time(input_df[system], series_length, window_len, step_size=rolling_step)
            for system in [first_random_system, second_random_system, third_random_system, 'WhiteNoise', 'SineWave']}
SaE_dict = {system: SaE_over_time(input_df[system], series_length, window_len, step_size=rolling_step, order = 3)
            for system in [first_random_system, second_random_system, third_random_system, 'WhiteNoise', 'SineWave']}

print("All Entropy values are calculated!")

In [ ]:
#This creates the index list for the final data-frame:
IndexList = input_df.index[::rolling_step] + window_len
IndexList = IndexList[IndexList < input_df.index[0] + series_length]


# This creates the dataframes of each set of entropy values over time for white noise, sine wave, and PV systems:
entropy_types = ['WPE', 'PE', 'DE', 'SpE', 'SaE']

for entropy_type in entropy_types:
    df = pd.DataFrame(globals()[entropy_type+'_dict'])
    df['newcol'] = IndexList
    df.index = pd.to_datetime(df['newcol'])
    df = df.drop(columns=['newcol'])
    globals()[entropy_type+'_df'] = df # one might like to save the generated results

In [ ]:
# This cell reads the pre-generated dataframes of each set of entropy values over time for white noise, sine wave,
# and PV systems (produced using the original dataset, which could not be shared)

WPE_df = pd.read_pickle("Processed_data/SinevsWNvsPVGen_WPE61.pkl")
PE_df = pd.read_pickle("Processed_data/SinevsWNvsPVGen_PE61.pkl")
DE_df = pd.read_pickle("Processed_data/SinevsWNvsPVGen_DE55.pkl")
SpE_df = pd.read_pickle("Processed_data/SinevsWNvsPVGen_SpE.pkl")
SaE_df = pd.read_pickle("Processed_data/SinevsWNvsPVGen_SaE3.pkl")

In [ ]:
# This cell plots the entropy values of 3 time series against those of a sine wave and a white noise (Figure 1)

import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 7
plt.rcParams['figure.dpi'] = 400
plt.rcParams['savefig.dpi'] = 400

cycler = plt.cycler(linestyle=['dashdot', 'dotted', 'dashed', (0, (1, 1)), 'solid'],
                    color=['darkorange', 'magenta','limegreen', 'darkred', 'darkgreen'])
plt.figure(figsize=(3.4, 7.2))

# Plotting the DE figure
ax = plt.subplot2grid((5, 1), (0, 0))
ax.set_prop_cycle(cycler)
ax.plot(DE_df, linewidth=0.75)
plt.legend(["PV generation 1", "PV generation 2", "PV generation 3",
           "White Gaussian noise", "Sine wave signal"], ncol=2,  fontsize=6)
plt.ylabel('DE')
plt.minorticks_off()
ax.axes.xaxis.set_ticklabels([])
plt.tick_params(axis='x', which='both', labelbottom=False)
plt.grid(color='gainsboro', linestyle='--', linewidth=0.4, alpha=0.5)
# for captions:
x0, xmax = plt.xlim()
y0, ymax = plt.ylim()
data_width = xmax - x0
data_height = ymax - y0
plt.text(x0 - data_width * 0.17, ymax - data_height*0.01, 'A', weight='bold',  fontsize=9)

# Plotting the PE figure
ax = plt.subplot2grid((5, 1), (1, 0))
ax.set_prop_cycle(cycler)
ax.plot(PE_df, linewidth=0.75)
plt.ylabel('PE')
plt.minorticks_off()
ax.axes.xaxis.set_ticklabels([])
plt.tick_params(axis='x', which='both', labelbottom=False)
plt.grid(color='gainsboro', linestyle='--', linewidth=0.4, alpha=0.5)
# for captions:
x0, xmax = plt.xlim()
y0, ymax = plt.ylim()
data_width = xmax - x0
data_height = ymax - y0
plt.text(x0 - data_width * 0.17, ymax - data_height*0.01, 'B', weight='bold',  fontsize=9)

# plotting the SaE figure
ax = plt.subplot2grid((5, 1), (2, 0))
ax.set_prop_cycle(cycler)
ax.plot(SaE_df, linewidth=0.75)
plt.ylabel('SaE')
plt.minorticks_off()
plt.minorticks_off()
ax.axes.xaxis.set_ticklabels([])
plt.tick_params(axis='x', which='both', labelbottom=False)
plt.grid(color='gainsboro', linestyle='--', linewidth=0.4, alpha=0.5)
# for captions:
x0, xmax = plt.xlim()
y0, ymax = plt.ylim()
data_width = xmax - x0
data_height = ymax - y0
plt.text(x0 - data_width * 0.17, ymax - data_height*0.01, 'C', weight='bold',  fontsize=9)

# Plotting the SpE figure
ax = plt.subplot2grid((5, 1), (3, 0))
ax.set_prop_cycle(cycler)
ax.plot(SpE_df, linewidth=0.75)
plt.ylabel('SpE')
plt.minorticks_off()
plt.minorticks_off()
ax.axes.xaxis.set_ticklabels([])
plt.tick_params(axis='x', which='both', labelbottom=False)
plt.grid(color='gainsboro', linestyle='--', linewidth=0.4, alpha=0.5)
# for captions:
x0, xmax = plt.xlim()
y0, ymax = plt.ylim()
data_width = xmax - x0
data_height = ymax - y0
plt.text(x0 - data_width * 0.17, ymax - data_height*0.01, 'D', weight='bold',  fontsize=9)

# Plotting the WPE figure
ax = plt.subplot2grid((5, 1), (4, 0))
ax.set_prop_cycle(cycler)
ax.plot(WPE_df, linewidth=0.75)
plt.ylabel('WPE')
plt.minorticks_off()
ax.axes.xaxis.set_ticklabels(
    ['Mar-19', '', 'May-19', '', 'Jul-19', '', 'Sep-19', '', 'Nov-19', '', 'Jan-20'])
plt.grid(color='gainsboro', linestyle='--', linewidth=0.4, alpha=0.5)
# for captions:
x0, xmax = plt.xlim()
y0, ymax = plt.ylim()
data_width = xmax - x0
data_height = ymax - y0
plt.text(x0 - data_width * 0.17, ymax - data_height*0.01, 'E', weight='bold',  fontsize=9)

plt.xlabel('Time, Month')
plt.subplots_adjust(wspace=0, hspace=0.13)
plt.show()
#plt.savefig('Figure1.pdf', dpi = 400, bbox_inches='tight')